# Tutorial 29: Orbital Mechanics

Kepler orbits and the two-body problem.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mechanics_dsl import PhysicsCompiler

compiler = PhysicsCompiler()

dsl_code = r"""
\system{orbit}
\defvar{r}{Radial distance}{m}
\defvar{phi}{True anomaly}{rad}
\parameter{mu}{1.0}{m^3/s^2}
\lagrangian{
    \frac{1}{2}*(\dot{r}^2 + r^2*\dot{phi}^2) + mu/r
}
\initial{r=1.0, phi=0.0, r_dot=0.0, phi_dot=1.2}
"""

result = compiler.compile_dsl(dsl_code)
sol = compiler.simulate(t_span=(0, 20), num_points=2000)

In [ ]:
t = sol['t']
r, phi = sol['y'][0], sol['y'][2]

# Cartesian
x = r * np.cos(phi)
y = r * np.sin(phi)

# Orbital elements
mu = 1.0
r_dot, phi_dot = sol['y'][1], sol['y'][3]
v_squared = r_dot**2 + (r*phi_dot)**2
energy = 0.5*v_squared - mu/r
a = -mu / (2*energy[0])  # Semi-major axis
h = r * r * phi_dot  # Angular momentum
e = np.sqrt(1 + 2*energy[0]*h[0]**2/mu**2)  # Eccentricity

print(f"Semi-major axis: a = {a:.3f}")
print(f"Eccentricity: e = {e:.3f}")
print(f"Orbit type: {'Ellipse' if e < 1 else 'Hyperbola' if e > 1 else 'Parabola'}")

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

axes[0].plot(x, y, 'b-', lw=1)
axes[0].plot(0, 0, 'yo', ms=20, label='Central body')
axes[0].set_xlabel('x')
axes[0].set_ylabel('y')
axes[0].set_title(f'Orbit (e={e:.2f})')
axes[0].axis('equal')
axes[0].grid(True, alpha=0.3)
axes[0].legend()

axes[1].plot(t, energy, 'r-')
axes[1].set_xlabel('Time')
axes[1].set_ylabel('Specific Energy')
axes[1].set_title('Energy Conservation')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()